<a href="https://colab.research.google.com/github/16Jayavelrajan/Chatbot/blob/main/HPC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update -y
!apt-get install -y openmpi-bin openmpi-common libopenmpi-dev


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,003 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,535 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd

In [ ]:
%%writefile mpi_hello.c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv); // Initialize MPI

    int world_rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &world_rank); // Get process rank

    int world_size;
    MPI_Comm_size(MPI_COMM_WORLD, &world_size); // Get total number of processes

    printf("Hello from processor %d out of %d processors!\n", world_rank, world_size);

    MPI_Finalize(); // Finalize MPI
    return 0;
}


Writing mpi_hello.c


In [ ]:
!mpicc -o mpi_hello mpi_hello.c


In [ ]:
!mpirun --allow-run-as-root --oversubscribe -np 4 ./mpi_hello


Hello from processor 3 out of 4 processors!
Hello from processor 0 out of 4 processors!
Hello from processor 1 out of 4 processors!
Hello from processor 2 out of 4 processors!


In [ ]:
%%writefile mpi_send_recv.c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);

    if (rank == 0) {
        int data = 42;
        MPI_Send(&data, 1, MPI_INT, 1, 0, MPI_COMM_WORLD);
        printf("Process 0 sent %d to Process 1\n", data);
    }
    else if (rank == 1) {
        int received_data;
        MPI_Recv(&received_data, 1, MPI_INT, 0, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
        printf("Process 1 received %d from Process 0\n", received_data);
    }

    MPI_Finalize();
    return 0;
}


Overwriting mpi_send_recv.c


In [ ]:
!mpicc -o mpi_send_recv mpi_send_recv.c
!mpirun --allow-run-as-root --oversubscribe -np 2 ./mpi_send_recv


Process 0 sent 42 to Process 1
Process 1 received 42 from Process 0


In [ ]:
%%writefile mpi_bsend.c
#include <mpi.h>
#include <stdio.h>
#include <stdlib.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);

    int buffer_size = MPI_BSEND_OVERHEAD + sizeof(int);
    void* buffer = malloc(buffer_size);
    MPI_Buffer_attach(buffer, buffer_size);

    if (rank == 0) {
        int data = 42;
        printf("Process 0: Sending %d to Process 1 using MPI_Bsend\n", data);
        MPI_Bsend(&data, 1, MPI_INT, 1, 0, MPI_COMM_WORLD);
    }
    else if (rank == 1) {
        int received_data;
        MPI_Recv(&received_data, 1, MPI_INT, 0, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
        printf("Process 1: Received %d from Process 0\n", received_data);
    }

    MPI_Buffer_detach(&buffer, &buffer_size);
    free(buffer);

    MPI_Finalize();
    return 0;
}


Writing mpi_bsend.c


In [ ]:
!mpicc -o mpi_bsend mpi_bsend.c
!mpirun --allow-run-as-root --oversubscribe -np 2 ./mpi_bsend


Process 0: Sending 42 to Process 1 using MPI_Bsend
Process 1: Received 42 from Process 0


In [ ]:
%%writefile mpi_nonblocking.c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);

    MPI_Request request;
    MPI_Status status;

    if (rank == 0) {
        int data = 42;
        printf("Process 0: Sending %d to Process 1 (Non-blocking)\n", data);
        MPI_Isend(&data, 1, MPI_INT, 1, 0, MPI_COMM_WORLD, &request);
        MPI_Wait(&request, &status);  // Ensuring completion before exit
    }
    else if (rank == 1) {
        int received_data;
        MPI_Irecv(&received_data, 1, MPI_INT, 0, 0, MPI_COMM_WORLD, &request);
        MPI_Wait(&request, &status);  // Ensure the message is received before printing
        printf("Process 1: Received %d from Process 0 (Non-blocking)\n", received_data);
    }

    MPI_Finalize();
    return 0;
}


Writing mpi_nonblocking.c


In [ ]:
!mpicc -o mpi_nonblocking mpi_nonblocking.c
!mpirun --allow-run-as-root --oversubscribe -np 2 ./mpi_nonblocking


Process 0: Sending 42 to Process 1 (Non-blocking)
Process 1: Received 42 from Process 0 (Non-blocking)


In [1]:
%%writefile mpi_init_finalize.c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    // Step 1: Initialize MPI
    MPI_Init(&argc, &argv);

    // Step 2: Get total number of processes
    int size;
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    // Step 3: Get process rank
    int rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);

    // Step 4: Print process details
    printf("Hello from process %d out of %d!\n", rank, size);

    // Step 5: Finalize MPI
    MPI_Finalize();

    return 0;
}



Writing mpi_init_finalize.c


In [2]:
!mpicc -o mpi_init_finalize mpi_init_finalize.c
!mpirun --allow-run-as-root --oversubscribe -np 4 ./mpi_init_finalize



Hello from process 1 out of 4!
Hello from process 3 out of 4!
Hello from process 2 out of 4!
Hello from process 0 out of 4!


In [3]:
%%writefile mpi_deadlock_avoid.c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);

    int data = 42;

    if (rank == 0) {
        MPI_Send(&data, 1, MPI_INT, 1, 0, MPI_COMM_WORLD);
        MPI_Recv(&data, 1, MPI_INT, 1, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
    }
    else if (rank == 1) {
        MPI_Recv(&data, 1, MPI_INT, 0, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
        MPI_Send(&data, 1, MPI_INT, 0, 0, MPI_COMM_WORLD);
    }

    printf("Process %d: Communication successful\n", rank);

    MPI_Finalize();
    return 0;
}


Writing mpi_deadlock_avoid.c


In [4]:
%%writefile mpi_nonblocking_deadlock_avoid.c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);

    int data = 42;
    MPI_Request request;

    if (rank == 0) {
        MPI_Isend(&data, 1, MPI_INT, 1, 0, MPI_COMM_WORLD, &request);
        MPI_Irecv(&data, 1, MPI_INT, 1, 0, MPI_COMM_WORLD, &request);
        MPI_Wait(&request, MPI_STATUS_IGNORE);
    }
    else if (rank == 1) {
        MPI_Isend(&data, 1, MPI_INT, 0, 0, MPI_COMM_WORLD, &request);
        MPI_Irecv(&data, 1, MPI_INT, 0, 0, MPI_COMM_WORLD, &request);
        MPI_Wait(&request, MPI_STATUS_IGNORE);
    }

    printf("Process %d: Communication successful (Non-blocking)\n", rank);

    MPI_Finalize();
    return 0;
}


Writing mpi_nonblocking_deadlock_avoid.c


In [5]:
!mpicc -o mpi_deadlock_avoid mpi_deadlock_avoid.c
!mpirun --allow-run-as-root --oversubscribe -np 2 ./mpi_deadlock_avoid


Process 1: Communication successful
Process 0: Communication successful


In [6]:
!mpicc -o mpi_nonblocking_deadlock_avoid mpi_nonblocking_deadlock_avoid.c
!mpirun --allow-run-as-root --oversubscribe -np 2 ./mpi_nonblocking_deadlock_avoid


Process 0: Communication successful (Non-blocking)
Process 1: Communication successful (Non-blocking)


In [7]:
%%writefile mpi_cartesian.c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    // Define a 2D grid of (rows x cols)
    int dims[2] = {2, 2};  // 2x2 grid (must match the number of processes)
    int periods[2] = {0, 0}; // No wraparound
    int coords[2];
    MPI_Comm cart_comm;

    // Create Cartesian topology
    MPI_Cart_create(MPI_COMM_WORLD, 2, dims, periods, 0, &cart_comm);

    // Get Cartesian coordinates of the current process
    MPI_Cart_coords(cart_comm, rank, 2, coords);

    printf("Process %d has Cartesian coordinates (%d, %d)\n", rank, coords[0], coords[1]);

    // Find neighboring ranks
    int left, right, up, down;
    MPI_Cart_shift(cart_comm, 0, 1, &up, &down);
    MPI_Cart_shift(cart_comm, 1, 1, &left, &right);

    printf("Process %d (Coords %d,%d) - Left: %d, Right: %d, Up: %d, Down: %d\n",
           rank, coords[0], coords[1], left, right, up, down);

    MPI_Finalize();
    return 0;
}


Writing mpi_cartesian.c


In [8]:
!mpicc -o mpi_cartesian mpi_cartesian.c
!mpirun --allow-run-as-root --oversubscribe -np 4 ./mpi_cartesian


Process 0 has Cartesian coordinates (0, 0)
Process 2 has Cartesian coordinates (1, 0)
Process 2 (Coords 1,0) - Left: -2, Right: 3, Up: 0, Down: -2
Process 0 (Coords 0,0) - Left: -2, Right: 1, Up: -2, Down: 2
Process 1 has Cartesian coordinates (0, 1)
Process 1 (Coords 0,1) - Left: 0, Right: -2, Up: -2, Down: 3
Process 3 has Cartesian coordinates (1, 1)
Process 3 (Coords 1,1) - Left: 2, Right: -2, Up: 1, Down: -2


In [9]:
%%writefile mpi_bcast.c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);

    int data;
    if (rank == 0) {
        data = 100; // Root process sets the data
        printf("Process 0 broadcasting data: %d\n", data);
    }

    MPI_Bcast(&data, 1, MPI_INT, 0, MPI_COMM_WORLD);
    printf("Process %d received data: %d\n", rank, data);

    MPI_Finalize();
    return 0;
}


Writing mpi_bcast.c


In [10]:
!mpicc -o mpi_bcast mpi_bcast.c
!mpirun --allow-run-as-root --oversubscribe -np 4 ./mpi_bcast


Process 0 broadcasting data: 100
Process 0 received data: 100
Process 2 received data: 100
Process 1 received data: 100
Process 3 received data: 100


In [11]:
%%writefile mpi_scan.c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    int local_value = rank + 1; // Each process has a unique value
    int prefix_sum;

    MPI_Scan(&local_value, &prefix_sum, 1, MPI_INT, MPI_SUM, MPI_COMM_WORLD);

    printf("Process %d: Prefix sum = %d\n", rank, prefix_sum);

    MPI_Finalize();
    return 0;
}


Writing mpi_scan.c


In [12]:
!mpicc -o mpi_scan mpi_scan.c
!mpirun --allow-run-as-root --oversubscribe -np 4 ./mpi_scan


Process 0: Prefix sum = 1
Process 1: Prefix sum = 3
Process 2: Prefix sum = 6
Process 3: Prefix sum = 10


In [13]:
%%writefile mpi_barrier.c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);

    printf("Process %d before barrier\n", rank);
    MPI_Barrier(MPI_COMM_WORLD);
    printf("Process %d after barrier\n", rank);

    MPI_Finalize();
    return 0;
}


Writing mpi_barrier.c


In [14]:
!mpicc -o mpi_barrier mpi_barrier.c
!mpirun --allow-run-as-root --oversubscribe -np 4 ./mpi_barrier


Process 0 before barrier
Process 2 before barrier
Process 1 before barrier
Process 3 before barrier
Process 0 after barrier
Process 1 after barrier
Process 3 after barrier
Process 2 after barrier
